In [ ]:
import nltk
from nltk.corpus import brown
#from nltk.tokenize import word_tokenize
import numpy as np
#import pandas as pd
import re
import copy
#import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import random #for shuffling list
from sklearn.model_selection import train_test_split
import tensorflow as tf
import gensim

nltk.download('punkt')
nltk.download('brown')
nltk.download('universal_tagset')
dataset = brown.tagged_sents(tagset='universal')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [ ]:
import gensim.downloader as api
import json
info = api.info()
print(json.dumps(info, indent=4))
print(api.load('word2vec-google-news-300', return_path=True))

In [ ]:
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz', binary=True)

In [ ]:
def unknown_word(word, train_set): #Cosine Similarity
    try:
        word_vec = w2v_model.wv[word]
    except:
        word_vec = np.random.rand(300)
    min = -1 #index
    value = -1 
    for i in range(len(train_set)):
        try:
            train_vec = w2v_model.wv[i]
        except:
            train_vec = np.random.rand(300)
        norm = np.linalg.norm(word_vec)*np.linalg.norm(train_vec)
        dot = np.dot(train_vec, word_vec)
        if dot/norm > value:
            value = dot/norm
            min = i
    return train_set[i]

In [ ]:

l = []

#sentences = list(brown.tagged_sents(tagset='universal'))
sentences = list(dataset)

for i in range(len(sentences)):
    test = sentences[i]
    test.insert(0, ("^", "START"))
    test.append(("$", "END"))
    sentences[i] = test
    for j in sentences[i]:
        l.append(j)
    
tags = {}
for i in l:
    if i[1] not in tags:
        tags[i[1]] = [i[0]]
    else:
        tags[i[1]].append(i[0])

words = {}
for i in l:
    wrd = i[0]
    if wrd not in words:
        words[wrd] = [i[1]]
    else:
        words[wrd].append(i[1])

tag_list = list(tags.keys())
print(tag_list)
exit()

#Calculating P(tag|word)
p_tag_given_word = {}
for tag in tag_list:
    for wrd in words:
        p_tag_given_word[( tag, wrd )] = words[wrd].count(tag)/len(words[wrd]) #Word with given tag count / total count of the word 'wrd' in the corpus

p_word_given_tag = {}
for tag in p_tag_given_word:
    p_word_given_tag[ tag[::-1] ] = p_tag_given_word[tag] * ( len(words[ tag[1] ])/len(l) ) / ( len(tags[ tag[0] ])/len(l) )

#Transition Probabilities P(tag -> tag)
p_tag_to_tag = {}

for item in sentences:
    for g in range(len(item)-1):
        if ( item[g][1], item[g+1][1] ) not in p_tag_to_tag:
            p_tag_to_tag[( item[g][1], item[g+1][1] )] = 1.0
        else:
            p_tag_to_tag[( item[g][1], item[g+1][1] )] += 1.0 #P(noun -> verb) += 1, we store count here

for t in p_tag_to_tag:
    p_tag_to_tag[t] /= len(tags[t[0]]) #P(noun -> verb) /= num(nouns)

for tag in tag_list:
    p_tag_to_tag[(tag, 'START')] = 0.0
    p_tag_to_tag[('END', tag)] = 0.0
p_tag_to_tag[('START', 'END')] = 0.0


#---------------------VITERBI-------------------------

actual_tags = []
predicted_tags = []

#print(len(all_sentences))
all_tags = tag_list 

transition_matrix=np.random.rand(len(all_tags),len(all_tags))
for i in range(len(all_tags)):
    for j in range(len(all_tags)):
        try:
            transition_matrix[i][j] = p_tag_to_tag[(all_tags[i], all_tags[j])]
        except:
            transition_matrix[i][j]=0.0
transition_matrix=transition_matrix.astype('float64')#(transition_matrix)

all_tokens = list(words.keys())

lexical_probs=np.random.rand(len(all_tags),len(all_tokens))
for i in range(len(all_tags)):
    for j in range(len(all_tokens)):
        try:
            lexical_probs[i][j] = p_word_given_tag[(all_tokens[j], all_tags[i])]
        except:
            lexical_probs[i][j]=0.0
lexical_probs=lexical_probs.astype('float64')#(lexical_probs)

#-------------------------------------------------------------------------------------
given_sent = "The long and lonely road to Mumbai."
token_ex = token_ex = re.findall(r"([\w'-]+|[.,!?\[\]\(\);:]|[`]+)", given_sent)
token_ex.insert(0, "^")
token_ex.append("$")

for index in range(len(token_ex)):
    wrd = token_ex[index]
    if not wrd in words:
        assign = unknown_word(wrd, list(words.keys())) #has cosine similarity calculated
        token_ex[index] = assign

prob_matrix=np.array([[0.0]*len(all_tags)]*len(all_tags))
for i in range(len(token_ex)-1):
    if i==0:
        prev_tag="START"
        transition_probs=np.array([transition_matrix[all_tags.index(prev_tag)][j] for j in range(len(all_tags))])
        obs_prob=lexical_probs[all_tags.index(prev_tag)][all_tokens.index(token_ex[i])]
        net_probs=obs_prob*transition_probs
        seq=[["START",all_tags[k]] for k in range(0,len(all_tags))]

    else:
        for k in range(0,len(all_tags)):
            transition_probs=np.array([transition_matrix[k][j] for j in range(len(all_tags))])
            obs_prob=lexical_probs[k][all_tokens.index(token_ex[i])]
            prob_matrix[k]=obs_prob*net_probs[k]*transition_probs

        new_seq=[[] for s in range(len(all_tags))]
        for k in range(0,len(all_tags)):
            a=prob_matrix[:,k].max
            idx=np.argmax(prob_matrix[:,k])
            new_seq[k]=copy.deepcopy(seq[idx])
            new_seq[k].append(all_tags[k])
            net_probs[k]=prob_matrix[idx][k]
        seq=copy.deepcopy(new_seq)

predicted = seq[1]
for i in range(len(token_ex)):
    print(token_ex[i], predicted[i])


['START', 'DET', 'NOUN', 'ADJ', 'VERB', 'ADP', '.', 'END', 'ADV', 'CONJ', 'PRT', 'PRON', 'NUM', 'X']
^ START
The DET
long ADJ
and CONJ
lonely ADJ
road NOUN
to PRT
stupefying VERB
. .
$ DET


In [ ]:
def sentence_decoding(training_sentences, testing_sentences):

    l = []

    #sentences = list(brown.tagged_sents(tagset='universal'))
    sentences = list(training_sentences)

    for i in range(len(sentences)):
        test = sentences[i]
        test.insert(0, ("^", "START"))
        test.append(("$", "END"))
        sentences[i] = test
        for j in sentences[i]:
            l.append(j)
        
    tags = {}
    for i in l:
        if i[1] not in tags:
            tags[i[1]] = [i[0]]
        else:
            tags[i[1]].append(i[0])

    words = {}
    for i in l:
        wrd = i[0]
        if wrd not in words:
            words[wrd] = [i[1]]
        else:
            words[wrd].append(i[1])

    tag_list = list(tags.keys())
    print(tag_list)
    exit()

    #Calculating P(tag|word)
    p_tag_given_word = {}
    for tag in tag_list:
        for wrd in words:
            p_tag_given_word[( tag, wrd )] = words[wrd].count(tag)/len(words[wrd]) #Word with given tag count / total count of the word 'wrd' in the corpus

    p_word_given_tag = {}
    for tag in p_tag_given_word:
        p_word_given_tag[ tag[::-1] ] = p_tag_given_word[tag] * ( len(words[ tag[1] ])/len(l) ) / ( len(tags[ tag[0] ])/len(l) )

    #Transition Probabilities P(tag -> tag)
    p_tag_to_tag = {}

    for item in sentences:
        for g in range(len(item)-1):
            if ( item[g][1], item[g+1][1] ) not in p_tag_to_tag:
                p_tag_to_tag[( item[g][1], item[g+1][1] )] = 1.0
            else:
                p_tag_to_tag[( item[g][1], item[g+1][1] )] += 1.0 #P(noun -> verb) += 1, we store count here

    for t in p_tag_to_tag:
        p_tag_to_tag[t] /= len(tags[t[0]]) #P(noun -> verb) /= num(nouns)

    for tag in tag_list:
        p_tag_to_tag[(tag, 'START')] = 0.0
        p_tag_to_tag[('END', tag)] = 0.0
    p_tag_to_tag[('START', 'END')] = 0.0
    

    #---------------------VITERBI-------------------------

    actual_tags = []
    predicted_tags = []

    #print(len(all_sentences))
    all_tags = tag_list 

    transition_matrix=np.random.rand(len(all_tags),len(all_tags))
    for i in range(len(all_tags)):
        for j in range(len(all_tags)):
            try:
                transition_matrix[i][j] = p_tag_to_tag[(all_tags[i], all_tags[j])]
            except:
                transition_matrix[i][j]=0.0
    transition_matrix=transition_matrix.astype('float64')#(transition_matrix)

    all_tokens = list(words.keys())

    lexical_probs=np.random.rand(len(all_tags),len(all_tokens))
    for i in range(len(all_tags)):
        for j in range(len(all_tokens)):
            try:
                lexical_probs[i][j] = p_word_given_tag[(all_tokens[j], all_tags[i])]
            except:
                lexical_probs[i][j]=0.0
    lexical_probs=lexical_probs.astype('float64')#(lexical_probs)

    for iter in range( len(testing_sentences) ):

        #given_sent = ' '.join([str(elem[0]) for elem in testing_sentences[iter]])
        actual_tag_sent = []
        actual_tag_sent = testing_sentences[iter]

        actual_tag_sent.insert(0, ("^", "START"))
        actual_tag_sent.append(("$", "END"))
        actual_tags.append([i[1] for i in actual_tag_sent])

        token_ex = [i[0] for i in actual_tag_sent]
        
        for index in range(len(token_ex)):
            wrd = token_ex[index]
            if not wrd in words:
                assign = unknown_word(wrd, list(words.keys())) #has cosine similarity calculated
                token_ex[index] = assign

        prob_matrix=np.array([[0.0]*len(all_tags)]*len(all_tags))
        for i in range(len(token_ex)-1):
            if i==0:
                prev_tag="START"
                transition_probs=np.array([transition_matrix[all_tags.index(prev_tag)][j] for j in range(len(all_tags))])
                obs_prob=lexical_probs[all_tags.index(prev_tag)][all_tokens.index(token_ex[i])]
                net_probs=obs_prob*transition_probs
                seq=[["START",all_tags[k]] for k in range(0,len(all_tags))]

            else:
                for k in range(0,len(all_tags)):
                    transition_probs=np.array([transition_matrix[k][j] for j in range(len(all_tags))])
                    obs_prob=lexical_probs[k][all_tokens.index(token_ex[i])]
                    prob_matrix[k]=obs_prob*net_probs[k]*transition_probs

                new_seq=[[] for s in range(len(all_tags))]
                for k in range(0,len(all_tags)):
                    a=prob_matrix[:,k].max
                    idx=np.argmax(prob_matrix[:,k])
                    new_seq[k]=copy.deepcopy(seq[idx])
                    new_seq[k].append(all_tags[k])
                    net_probs[k]=prob_matrix[idx][k]
                seq=copy.deepcopy(new_seq)

        predicted_tags.append(seq[1])
    
    actual_total=[]
    predicted_total=[]
    for i in range(len(actual_tags)):
        for j in range(len(actual_tags[i])):
            actual_total.append(actual_tags[i][j])
            predicted_total.append(predicted_tags[i][j])
    
    print(confusion_matrix(actual_total, predicted_total))
    return actual_tags, predicted_tags

In [ ]:
def accuracy_per_fold(training_sentences, test_sentences):
    test_y,predictions = sentence_decoding(training_sentences, test_sentences)

    f1 = 0
    prec = 0
    recall = 0
    

    #for i in range( len(actual_tags) ):
    #    f1 += f1_score(actual_tags[i], predicted_tags[i], average='weighted')
    #    prec += precision_score(actual_tags[i], predicted_tags[i], average='weighted')
    #    recall += recall_score(actual_tags[i], predicted_tags[i], average='weighted')
    per_pos=[[] for i in range(12)]
    #per_pos_act=[[] for i in range(0,12)]
    for i in range(len(test_y)):
        per_pos[test_y[i]].append(predictions[i])
    print("my_fold------------------------")
    for i in range(len(per_pos)):
        print(i,"-----")
        true_tag=[i for j in range(len(per_pos[i]))]
        print("precision",precision_score(per_pos[i],true_tag, average='weighted'))
        print("recall",recall_score(per_pos[i],true_tag, average='weighted'))
        print("f1",f1_score(per_pos[i],true_tag, average='weighted'))
    print("overall for the batch:")
    precision=precision_score(test_y,predictions, average='weighted')
    recall=recall_score(test_y,predictions, average='weighted')
    f1=f1_score(test_y,predictions, average='weighted')
    print("precision",precision)
    print("recall",recall)
    print("f1",f1)

        
    return [precision, recall, f1]
    #return [f1/len(actual_tags), prec/len(actual_tags), recall/len(actual_tags)]

#total_acc_for_split1 = accuracy_per_fold(all_sentences[:57000], all_sentences[56000:56010])
#total_acc_for_split2 = accuracy_per_fold(all_sentences[:57000], all_sentences[57000:57010])

#print("\nTotal accuracy for the above split1: " + str(total_acc_for_split1))
#print("Total accuracy for the above split2: " + str(total_acc_for_split2) + "\n")
#exit(0)

def cross_validation_accuracy(all_sentences = dataset):
    all_accuracies = []
    total_acc = [0, 0, 0]

    num_sentences = len(dataset)

    fold_size = int(num_sentences / 5)

    print(fold_size)

    fold1_train=dataset[:4*fold_size]
    fold1_test=dataset[4*fold_size:]
    fold2_train=dataset[:3*fold_size]+dataset[4*fold_size:]
    fold2_test=dataset[3*fold_size:4*fold_size]
    fold3_train=dataset[:2*fold_size]+dataset[3*fold_size:]
    fold3_test=dataset[2*fold_size:3*fold_size]
    fold4_train=dataset[:1*fold_size]+dataset[2*fold_size:]
    fold4_test=dataset[1*fold_size:2*fold_size]
    fold5_train=dataset[1*fold_size:]
    fold5_test=dataset[:1*fold_size]

    all_accuracies.append(accuracy_per_fold(fold1_train, fold1_test))
    print("Accuracy for fold ", "1", " is : ", str(all_accuracies[0]))
    total_acc[0] += all_accuracies[0][0]
    total_acc[1] += all_accuracies[0][1]
    total_acc[2] += all_accuracies[0][2]


    all_accuracies.append(accuracy_per_fold(fold2_train, fold2_test))
    print("Accuracy for fold ", "2", " is : ", str(all_accuracies[1]))
    total_acc[0] += all_accuracies[1][0]
    total_acc[1] += all_accuracies[1][1]
    total_acc[2] += all_accuracies[1][2]

    all_accuracies.append(accuracy_per_fold(fold3_train, fold3_test))
    print("Accuracy for fold ", "3", " is : ", str(all_accuracies[2]))
    total_acc[0] += all_accuracies[2][0]
    total_acc[1] += all_accuracies[2][1]
    total_acc[2] += all_accuracies[2][2]

    all_accuracies.append(accuracy_per_fold(fold4_train, fold4_test))
    print("Accuracy for fold ", "4", " is : ", str(all_accuracies[3]))
    total_acc[0] += all_accuracies[3][0]
    total_acc[1] += all_accuracies[3][1]
    total_acc[2] += all_accuracies[3][2]

    all_accuracies.append(accuracy_per_fold(fold5_train, fold5_test))
    print("Accuracy for fold ", "5", " is : ", str(all_accuracies[4]))
    total_acc[0] += all_accuracies[4][0]
    total_acc[1] += all_accuracies[4][1]
    total_acc[2] += all_accuracies[4][2]

    print("Cross validation accuracy is : ", str(np.array(total_acc) / 5))

In [ ]:
cross_validation_accuracy()

11468
['START', 'DET', 'NOUN', 'ADJ', 'VERB', 'ADP', '.', 'END', 'ADV', 'CONJ', 'PRT', 'PRON', 'NUM', 'X']


NameError: ignored

In [ ]:
test_y[0]

NameError: ignored